## 1. Setup

In [ ]:
# Install required libraries (run this once if needed)
%pip install numpy pandas matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 2. Dataset and Notation

- M: stellar mass (in units of solar mass, M⊙)
- T: effective stellar temperature (Kelvin, K)
- L: stellar luminosity (in units of solar luminosity, L⊙

M = [0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4]

L = [0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0]

In [ ]:
M = np.array([0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])
L = np.array([0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0])

m = len(M)
l = len(L)

### 2.1 DataSet Visualization: Plot M vs L. 



In [ ]:
plt.figure()
plt.scatter(M, L)
plt.xlabel("Stellar Mass (M☉)")
plt.ylabel("Luminosity (L☉)")
plt.title("Stellar Luminosity vs Mass")
plt.grid(True)
plt.show()


The plot shows a strong positive correlation between stellar mass and luminosity, that is a good sign of some plausibility.

However, the relationship between both is clearly non linear: 
- luminosity increases slowly at low masses and much more rapidly at higher masses.


This suggests that a linear regression model will only provide a rough approximation and will underpredict luminosity for high-mass stars, as the relationship is not linear at higher masses.

## 3.Model and loss

The hypothesis models stellar luminosity as a linear function of mass with an explicit bias term.

The mean squared error is used as the loss function, measuring the average squared difference between predicted and observed luminosities.


Prediction

In [ ]:
def predict(M, w, b):
    """Compute predictions f_{w,b}(x) for all examples.
    
    Linear regression model:
    L_hat = w * M + b
    
    """
    return M * w + b  # vectorized: matrix-vector product + scalar

w_test = 0.0
b_test = 0.0

l_hat_test = predict(M, w_test, b_test)
print("First 3 predictions with w = 0, b = 0:", l_hat_test[:3])

MSE: mean squared error

In [ ]:
## def compute_cost(x, y, w, b):
#    m = len(x)
#    y_hat = w * x + b
#    return np.sum((y_hat - y) ** 2) / (2 * m)

In [ ]:
def mse(M, L, w, b):  #compute_cost
    """
    Mean Squared Error cost function
    """
    m = len(M)
    L_hat = predict(M, w, b)
    return (1 / (2 * m)) * np.sum((L_hat - L) ** 2) 

print("Cost with w = 0, b = 0: ", mse(M, L, w_test, b_test))

## 4. Cost surface

Visualizing the cost surface before applying methods like gradient descent helps us understand the behavior of the model.
It allows us to see that there is a single global minimum, which is important to ensure that gradient descent will converge correctly.

Additionally, it helps us understand the sensitivity of the parameters: how small changes in w or b affect the cost.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # needed to register the 3D projection
from matplotlib import cm
 
# Choose reasonable ranges around the expected optimum
w_vals = [float(v) for v in np.linspace(-1.0, 7.0, 60)]
b_vals = [float(v) for v in np.linspace(-5.0, 10.0, 60)]
 
J = np.zeros((len(w_vals), len(b_vals)))
 
for i, w in enumerate(w_vals):
    for j, b in enumerate(b_vals):
        J[i, j] = mse(M, L, w, b)
W, B = np.meshgrid(w_vals, b_vals)
 
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(W, B, J, cmap=cm.viridis, linewidth=0, antialiased=True)
ax.set_xlabel("w")
ax.set_ylabel("b")
ax.set_zlabel("J(w,b)")
ax.set_title("Cost surface J(w,b)")
plt.show()

The 3D plot shows how the MSE varies with slope (w) and bias (b).
The lowest point on the surface represents the optimal parameters that best fit the data.
Its convex shape confirms that the cost function has a single global minimum, which is why gradient descent can reliably find the optimal w and b.

## 5. Gradients

$$
\frac{\partial J}{\partial w} = \frac{1}{m} \sum_{i=1}^{m} \big( f_{w,b}(x^{(i)}) - y^{(i)} \big) x^{(i)}
$$

$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} \big( f_{w,b}(x^{(i)}) - y^{(i)} \big)
$$


$$
w := w - \alpha \frac{\partial J}{\partial w}, \qquad
b := b - \alpha \frac{\partial J}{\partial b}
$$

### 5.1 Gradient descent (non-vectorized)

In this implementation, gradients are computed using explicit loops over the dataset.
This approach closely follows the mathematical definition of gradient descent and is useful for understanding how parameter updates are performed step by step, although it is computationally inefficient for large datasets.

In [ ]:

def compute_gradients(M, L, w, b):
    m = len(M)
    sum_dw = 0.0
    sum_db = 0.0


    for i in range (m):
        error = (w * M[i] + b) - L[i]

        sum_dw += error * M[i]
        sum_db += error

    sum_dw /= m
    sum_db /= m
    return sum_dw, sum_db

dw_test, db_test = compute_gradients(M, L, w_test, b_test)
print("dw =", dw_test, "db =", db_test) 


### 5.2 Gradient descent (vectorized)

This implementation computes gradients using NumPy vectorized operations instead of explicit loops.
While mathematically equivalent to the non-vectorized version, this approach is significantly more efficient and scales better to large datasets.

In [ ]:
def compute_gradients_vectorized(M, L, w, b):
    m = len(M)
    error = (w * M + b) - L

    dj_dw = (1 / m) * np.sum(error * M) 
    dj_db = (1 / m) * np.sum(error)
    return dj_dw, dj_db

dj_dw_test, dj_db_test = compute_gradients(M, L, w_test, b_test)
print("Gradients at w=0, b=0:", dj_dw_test, dj_db_test)

### 5.3 Gradient descent (vectorized)


In this implementation, the gradients are computed using NumPy vectorized operations without explicit loops over the dataset.
This approach is mathematically equivalent to the non-vectorized version but is significantly more efficient and scalable.


$$
w := w - \alpha \frac{\partial J}{\partial w}, \qquad
b := b - \alpha \frac{\partial J}{\partial b}
$$

In [ ]:
def gradient_descent_vectorized(M, L, alpha, num_iters):
    w = 0.0
    b = 0.0

    for _ in range(num_iters):
        dw, db = compute_gradients_vectorized(M, L, w, b)
        w -= alpha * dw
        b -= alpha * db

    return w, b


### 5.4 Implement the Gradient Descent Loop

In [ ]:

def gradient_descent(M, L, w_init, b_init, alpha, num_iterations):
    w = w_init
    b = b_init
    history = []
    
    for i in range(num_iterations):
        dj_dw, dj_db = compute_gradients_vectorized(M, L, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        cost = mse(M, L, w, b)
        history.append((i, cost))

        if i % max(1, (num_iterations // 10)) == 0:
            print(f"Iteration {i:4d}: w={w:7.4f}, b={b:7.4f}, cost={cost:8.4f}")

    return w, b, history



alpha = 0.01
num_iterations = 2000

w_init = 0.0
b_init = 0.0

w_learned, b_learned, history = gradient_descent(M, L, w_init, b_init, alpha, num_iterations)
print("\nLearned parameters:")
print("w =", w_learned)
print("b =", b_learned)


## 6. Convergence (mandatory) Plot the Cost over Iterations

In [ ]:
plt.figure()
plt.plot(history)
plt.xlabel("Iteration")
plt.ylabel("Cost J(w,b)")
plt.title("Convergence of Gradient Descent: Cost vs Iterations")
plt.show()

The cost decreases monotonically as the number of iterations increases, indicating successful convergence.
With the chosen learning rate, the optimization is stable and does not show oscillations or divergence.
Most of the reduction in cost occurs during the first iterations, after which the improvement becomes gradual.